# [Raiffeisen Data Cup](https://boosters.pro/champ_11)

Клиенты Райффайзенбанка совершают покупки и снимают наличные в банкоматах с помощью карточек. Получив в виде обезличенных данных их историю транзакций, информацию о мерчантах (место, позволяющее принимать платежи с использованием банковской пластиковой карты), участники чемпионата должны предсказать две пары координат: дом и работу клиента. Оценкой качества решения в задаче является процент попаданий в окружность радиуса 0.02 градуса относительно реальных координат дома и работы.

# Loading Stuff

In [0]:
!pip install -q lightgbm
import pandas as pd
import numpy as np
import datetime
from lightgbm import LGBMClassifier
import sklearn

from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('train.csv')
df.rename(columns={"pos_adress_lat": "pos_address_lat","pos_adress_lon": "pos_address_lon"}, inplace=True)
df_test = pd.read_csv('test.csv')

df['is_train'] = np.int32(1)
df_test['is_train'] = np.int32(0)
dt_init = pd.concat([df, df_test])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
submission = pd.DataFrame(df_test['customer_id'].unique(), columns = ['_ID_'])

# Data Cleaning

- У заграничных транзакций нет ни адреса, ни координат; у некоторых есть координаты дома/работы, но этот список не пересекается со списком транзакций без указания координат работы — их нужно дропнуть
- Поскольку atm_address(_lat,_lon) и pos_address(_lat, _lon) не пересекаются, можно скопировать atm_address в pos_address и добавить признак is_atm

In [0]:
def clean(df_orig):
  # unify_address
  df = df_orig.copy()
  atm = df.atm_address_lat.notnull()
  df['is_atm'] = np.zeros(len(df), dtype=int)
  df.loc[atm, 'is_atm'] = 1
  df.loc[atm, 'pos_address_lat'] = df[atm].atm_address_lat
  df.loc[atm, 'pos_address_lon'] = df[atm].atm_address_lon
  df.loc[df.atm_address.notnull(), 'pos_address'] = df[df.atm_address.notnull()].atm_address
  df.drop(['atm_address', 'atm_address_lat', 'atm_address_lon'], axis=1, inplace=True)
  
  
  # fix country
  df.loc[df.country == 'RU', 'country'] = 'RUS'
  df.loc[df.country == 'RU ', 'country'] = 'RUS'
  
  # drop missing
  df.drop(df.index[df.pos_address_lat.isnull()], inplace=True)
  df.drop(df.index[df.terminal_id.isnull()], inplace=True)
  df.drop(df.index[df.transaction_date.isnull()], inplace=True)
  
  def clean_mcc(mcc):
    if type(mcc) == int:
        return mcc
    mcc = mcc.split(',')
    if len(mcc) == 1:
        return int(mcc[0])
    else:
        return 1000*int(mcc[0]) + int(mcc[1])
  df['mcc'] = df['mcc'].apply(clean_mcc)
   
  return df
  

dt_clean = clean(dt_init)

# Feature Engineering

### Идеи

- [ ] нормализовать города
- [x] groupby(cid, [lon, lat, mcc, mcc_group, terminal_id])
- [x] почистить данные по координатам
- [x] даты: выходные, праздники, дни недели
- [ ] предсказывание координаты/расстояния
- [x] аггрегирование признаков по cid
- [x] аггрегирование предсказаний для клиента
- [x] доля чекинов в данном терминале
- [x] доля чекинов в выходные

Не зашло:

- замена pos_terminal_lat/lon на медиану/среднее по terminal_id
- разные вариации на тему среднего чека
- разные вариации на тему расстояния до медианной/средней точки
- доля транзакций в данном городе

In [0]:
def features(dt_orig):
  dt = dt_orig.copy()
  # Deal with categorial features
  dt['city'] = dt['city'].factorize()[0].astype(np.int32)
  dt['country'] = dt['country'].factorize()[0].astype(np.int32)
  dt['address'] = dt['pos_address_lat'].apply(lambda x: "%.02f" % x) + ';' + dt['pos_address_lon'].apply(lambda x: "%.02f" % x)
  dt['address'] = dt['address'].factorize()[0].astype(np.int32)
  
  # Add is_home and is_work target
  lat = dt['home_add_lat'] - dt['pos_address_lat']
  lon = dt['home_add_lon'] - dt['pos_address_lon']
  dt['is_home'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
  dt['has_home'] = (~dt['home_add_lon'].isnull()).astype(np.int32)

  lat = dt['work_add_lat'] - dt['pos_address_lat']
  lon = dt['work_add_lon'] - dt['pos_address_lon']
  dt['is_work'] = (np.sqrt((lat ** 2) + (lon ** 2)) <= 0.02).astype(np.int32)
  dt['has_work'] = (~dt['work_add_lon'].isnull()).astype(np.int32)
  
  # количество транзакций каждого клиента
  dt = dt.merge(dt.groupby('customer_id')['amount'].count().reset_index(name = 'tx'), how = 'left')
  dt['tx'] = dt['tx'].astype(np.int32)

  # количество транзакций каждого клиента с данным адресом
  dt = dt.merge(dt.groupby(['customer_id','address'])['amount'].count().reset_index(name = 'tx_cust_addr'), how = 'left')
  dt['tx_cust_addr'] = dt['tx_cust_addr'].astype(np.int32)

  # какая часть транзакций клиента приходится на данный адрес
  dt['ratio1'] = dt['tx_cust_addr'] / dt['tx']
  
  # день недели, выходной/нет
  dt['transaction_date'] = dt['transaction_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
  dt['weekday'] = dt['transaction_date'].dt.weekday.astype(np.int32)
  dt['weekend'] = (dt.weekday >= 5).factorize()[0].astype(np.int32)
  
  # количество чекинов в выходные данного клиента в данной точке
  dt = dt.merge(dt.groupby(['customer_id','address','weekend'])['amount'].count().reset_index(name = 'tx_cust_addr_weekend'), how = 'left')
  dt['tx_cust_addr_weekend'] = dt['tx_cust_addr_weekend'].astype(np.int32)
  
  # какая часть транзакций клиента на данный адрес приходится на выходные
  dt['ratio2'] = dt['tx_cust_addr_weekend'] / dt['tx']
  dt['ratio3'] = dt['tx_cust_addr_weekend'] / dt['tx_cust_addr']
  
  # какая часть транзакций клиента приходится на этот город
  dt = dt.merge(dt.groupby(['customer_id','city'])['amount'].count().reset_index(name = 'tx_cust_city'), how = 'left')
  dt['ratio4'] = dt['tx_cust_city'] / dt['tx']
  
  # группа кодов mcc
  dt['mcc_group'] = dt.mcc // 100
  
  return dt

dt = features(dt_clean)

In [0]:
def _best(x):
    ret = None
    for col in ys:
        pred = ('pred:%s' % col)
        if pred in x:
            i = (x[pred].idxmax())
            cols = [pred,'pos_address_lat','pos_address_lon']
            if col in x:
                cols.append(col)
            tmp = x.loc[i,cols]
            tmp.rename({
                'pos_address_lat':'%s:add_lat' % col,
                'pos_address_lon':'%s:add_lon' % col,
            }, inplace = True)
            if ret is None:
                ret = tmp
            else:
                ret = pd.concat([ret, tmp])
    return ret
  
def predict_proba(dt, ys = ['is_home', 'is_work']):
    for col in ys:
        pred = ('pred:%s' % col)
        dt[pred] = model[col].predict_proba(dt[xs])[:,1]
    return dt.groupby('customer_id').apply(_best).reset_index()
  
def score(dt, ys = ['is_home', 'is_work']):
    dt_ret = predict_proba(dt, ys)
    mean = 0.0
    for col in ys:
        col_mean = dt_ret[col].mean()
        mean += col_mean
    if len(ys) == 2:
        mean = mean / len(ys)
    return mean

# LGBM

In [134]:
xs = ['amount','currency','city','country','mcc','is_atm','ratio1','weekday',
      'weekend','ratio2','ratio3','mcc_group']
ys = ['is_home', 'is_work']

model0 = {
    'is_home': LGBMClassifier(),
    'is_work': LGBMClassifier(),
}

model = {}

# последовательно обучаем два классификатора
for col in ['is_home', 'is_work']:
    
    # выберем для обучения транзакции только тех клиентов из train, у которых хоть в одной транзакции указано место работы/жительства
    cust_train = dt[dt['is_train'] == 1].groupby('customer_id')[col.replace('is_','has_')].max()
    cust_train = cust_train[cust_train > 0].index
    
    # разобъем train на train/valid для валидации
    cust_train, cust_valid = train_test_split(cust_train, test_size = 0.1, shuffle = True, random_state = 2)
    
    train = pd.DataFrame(cust_train, columns = ['customer_id']).merge(dt, how = 'left')
    valid = pd.DataFrame(cust_valid, columns = ['customer_id']).merge(dt, how = 'left')

    print ("Training:", col)
    clf = sklearn.base.clone(model0[col])
    clf.fit(train[xs], train[col], eval_metric = 'logloss', eval_set = [(train[xs], train[col]), (valid[xs], valid[col])], verbose=10)
    model[col] = clf
    print ("Train accuracy:", score(train, ys = [col]))
    print ("Test accuracy:", score(valid, ys = [col]))
    print ()

Training: is_home
[10]	valid_0's binary_logloss: 0.501454	valid_1's binary_logloss: 0.50261
[20]	valid_0's binary_logloss: 0.459504	valid_1's binary_logloss: 0.463002
[30]	valid_0's binary_logloss: 0.446354	valid_1's binary_logloss: 0.453868
[40]	valid_0's binary_logloss: 0.440367	valid_1's binary_logloss: 0.450772
[50]	valid_0's binary_logloss: 0.435478	valid_1's binary_logloss: 0.44917
[60]	valid_0's binary_logloss: 0.431784	valid_1's binary_logloss: 0.448543
[70]	valid_0's binary_logloss: 0.428658	valid_1's binary_logloss: 0.447849
[80]	valid_0's binary_logloss: 0.426075	valid_1's binary_logloss: 0.447657
[90]	valid_0's binary_logloss: 0.424233	valid_1's binary_logloss: 0.447195
[100]	valid_0's binary_logloss: 0.421783	valid_1's binary_logloss: 0.447114
Train accuracy: 0.41788888888888887
Test accuracy: 0.403

Training: is_work
[10]	valid_0's binary_logloss: 0.438936	valid_1's binary_logloss: 0.45402
[20]	valid_0's binary_logloss: 0.382058	valid_1's binary_logloss: 0.410363
[30]	val

In [0]:
cust_test = dt[dt['is_train'] == 0]['customer_id'].unique()
test = pd.DataFrame(cust_test, columns = ['customer_id']).merge(dt, how = 'left')
test = predict_proba(test)
test.rename(columns = {
        'customer_id':'_ID_',
        'is_home:add_lat': '_HOME_LAT_',
        'is_home:add_lon': '_HOME_LON_',
        'is_work:add_lat': '_WORK_LAT_',
        'is_work:add_lon': '_WORK_LON_'}, inplace = True)
test = test[['_ID_', '_WORK_LAT_', '_WORK_LON_', '_HOME_LAT_', '_HOME_LON_']]

In [0]:

# Заполняем пропуски
submission2 = submission.merge(test, how = 'left').fillna(0)

# Пишем файл submission
submission2.to_csv('submission.csv', index = False)